In [ ]:
from telegram import Update
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext
import random
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import logging
from typing import List

# Объявите токены и подключитесь к Google Sheets
TELEGRAM_API_TOKEN = 'TELEGRAM_TOKEN'
GSHEETS_API_TOKEN = 'C:/Users/Administrator/Documents/linkedin-390312-a3dddc27826d.json'
SPREADSHEET_NAME = 'LinkedIn Network'
URLS_LIST_SHEET_NAME = 'URLs list'
USER_LINKS_HISTORY_SHEET_NAME = 'Linkedin Network user links history'

# Подключение к таблице
scope = ["https://spreadsheets.google.com/feeds", 'https://www.googleapis.com/auth/spreadsheets',
         "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive"]

creds = ServiceAccountCredentials.from_json_keyfile_name(GSHEETS_API_TOKEN, scope)
client = gspread.authorize(creds)
spreadsheet = client.open(SPREADSHEET_NAME)
urls_list_sheet = spreadsheet.worksheet(URLS_LIST_SHEET_NAME)
user_links_history_sheet = None
sent_links_history_sheet = spreadsheet.worksheet(USER_LINKS_HISTORY_SHEET_NAME)


# Проверка существования листа
existing_sheets = [sheet.title for sheet in spreadsheet.worksheets()]
if USER_LINKS_HISTORY_SHEET_NAME in existing_sheets:
    user_links_history_sheet = spreadsheet.worksheet(USER_LINKS_HISTORY_SHEET_NAME)
else:
    user_links_history_sheet = spreadsheet.add_worksheet(
        title=USER_LINKS_HISTORY_SHEET_NAME, rows="100", cols="3"
    )


# Настройка логирования
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO
)
logger = logging.getLogger(__name__)


def start(update: Update, context: CallbackContext) -> None:
    """Send a welcome message and provide command help."""
    user_id = str(update.effective_user.id)
    update.message.reply_text(
        f"Hello, {update.effective_user.first_name}!\n"
        "Send me a link to your LinkedIn profile, and I will provide you with "
        "several random profiles for networking."
    )
    context.user_data['add'] = True  # Устанавливаем состояние пользователя на "добавление"


def process_link(update: Update, context: CallbackContext) -> None:
    """Process the incoming link."""
    user_id = str(update.effective_user.id)
    text = update.message.text
    sent_links = get_sent_links(user_id)

    if 'add' in context.user_data and context.user_data['add']:
        context.user_data.pop('add', None)
        add_link(user_id, text, update, context, sent_links)
    elif 'delete' in context.user_data and context.user_data['delete']:
        context.user_data.pop('delete', None)
        delete_link(update, context)
    else:
        update.message.reply_text("I don't understand what you want to do with this link.")


def add(update: Update, context: CallbackContext) -> None:
    """Set the user state to 'add'."""
    context.user_data['add'] = True
    update.message.reply_text('Send me the LinkedIn link you want to add.')


def delete(update: Update, context: CallbackContext) -> None:
    """Set the user state to 'delete'."""
    context.user_data['delete'] = True
    update.message.reply_text('Send me the LinkedIn link you want to delete.')
    
def get_user_profile_url(user_id: str) -> str:
    user_links = get_user_links(user_id)
    if user_links:
        return user_links[0]  # Первая ссылка, которую отправил пользователь, предположительно его профиль
    return None

def get_all_links() -> List[str]:
    all_links_data = urls_list_sheet.get_all_values()
    all_links = [row[1] for row in all_links_data]  # Assuming the URL is in the second column
    return all_links


def get_new_links(user_id: str, already_sent_links: List[str]) -> List[str]:
    all_links = get_all_links()
    new_links = list(set(all_links) - set(already_sent_links))
    return new_links

def get_id_by_link(link: str) -> int:
    all_links = urls_list_sheet.get_all_values()
    for row in all_links:
        if row[1] == link:
            return int(row[0])  # returning the id of the link
    return None



def show_more(update: Update, context: CallbackContext) -> None:
    """Показать пользователю больше ссылок."""
    user_id = str(update.effective_user.id)
    user_links = get_user_links(user_id)
    sent_links = get_sent_links(user_id)
    
    # Проверяем, были ли уже отправлены ссылки пользователю. Если нет, выводим сообщение.
    if not sent_links:
        update.message.reply_text('You have not received any links yet.')
        return

    # Берем больше ссылок для отправки пользователю.
    bot_links = get_random_bot_links(user_id, user_links, sent_links, 5)

    # Обновляем историю ссылок пользователя.
    update_user_links_history(user_id, user_links, bot_links)

    # Отправляем новые ссылки пользователю.
    text = "Here are some more profiles for you to connect with:\n\n"
    text += "\n".join(bot_links)
    text += "\n\nIf you want more, use /show_more."
    update.message.reply_text(text)
    
    # After sending links to the user
    sent_link_ids = [get_id_by_link(link) for link in bot_links]  # Convert URLs to IDs
    update_sent_links_history(user_id, sent_link_ids)




def delete_new_user_links_history(user_id: str, start_row: int, end_row: int) -> None:
    """Delete new rows from the links history."""
    user_links_history_sheet.delete_rows(start_row, end_row)


def delete_user_links(user_id: str) -> None:
    """Delete all user links from the links history."""
    user_links_history = user_links_history_sheet.get_all_values()
    indices_to_delete = []
    for i, row in enumerate(user_links_history, start=1):
        if row[0] == user_id:
            indices_to_delete.append(i)
    for index in reversed(indices_to_delete):
        user_links_history_sheet.delete_rows(index)


def get_max_id(all_links):
    max_id = 0
    for row in all_links:
        link_id = int(row[0])
        if link_id > max_id:
            max_id = link_id
    return max_id


def get_all_user_links(user_id: str) -> List[str]:
    user_links_history = user_links_history_sheet.get_all_values()
    for row in user_links_history:
        if row[0] == user_id:
            return row[1:]
    return []


def get_link_id(link: str) -> str:
    all_links = urls_list_sheet.get_all_values()
    for row in all_links:
        if row[1] == link:  # if the URL in the row matches the link
            return row[0]  # return the ID of the link
    return None

def get_link_by_id(id: int) -> str:
    all_links = urls_list_sheet.get_all_values()
    for row in all_links:
        if int(row[0]) == id:  # Проверяем, что идентификатор в строке совпадает с искомым
            return row[1]  # Возвращаем соответствующую ссылку
    return None


def add_link(user_id: int, url: str, update: Update, context: CallbackContext, sent_links: List[str]) -> None:
    user_links = get_user_links(user_id)

    # Если команда /add в состоянии пользователя, удаляем ее
    if 'add' in context.user_data:
        context.user_data.pop('add')

    # если ссылка уже есть в списке пользователя, возвращаемся
    if url in user_links:
        update.message.reply_text("This link is already in your list.")
        return

    # иначе, добавляем ссылку в список пользователя
    user_links.append(url)
    bot_links = get_random_bot_links(user_id, url, user_links, sent_links, 5)

    # добавляем новую ссылку в общий список, если ее там нет
    all_links = urls_list_sheet.get_all_values()
    all_links_urls = [row[1] for row in all_links]
    if url not in all_links_urls:
        urls_list_sheet.insert_row([str(user_id), url], len(all_links) + 1)

    # Convert bot_links (IDs) to URLs
    bot_links = [get_link_by_id(link_id) for link_id in bot_links]

    # Remove user's own profile link from the list
    bot_links = [link for link in bot_links if link != url]

    # отправляем ответ пользователю
    text = "Here are some profiles for you to connect with:\n\n"
    text += "\n".join(bot_links)
    text += "\n\nIf you want more, use /show_more."
    update.message.reply_text(text)

    # After sending links to the user, update sent links history
    update_sent_links_history(user_id, bot_links)







def delete_link(update: Update, context: CallbackContext) -> None:
    user_id = str(update.effective_user.id)
    text = update.message.text
    user_links = get_user_links(user_id)

    if len(user_links) == 0:
        update.message.reply_text('You have no links to delete.')
        return

    if text in user_links:
        user_links.remove(text)
        delete_user_link(user_id, text)

        # Delete the link from the overall list
        all_links = urls_list_sheet.get_all_values()
        for i, row in enumerate(all_links, start=1):  # enumerate with start=1 to match the 1-indexing of rows in the sheet
            if row[1] == text:
                urls_list_sheet.delete_rows(i)

        update.message.reply_text('The link has been successfully deleted from the database.')
    else:
        update.message.reply_text('You do not have such a link in the database.')


def delete_user_link(user_id: str, link: str) -> None:
    """Удалить ссылку пользователя из истории ссылок."""
    user_links_history = user_links_history_sheet.get_all_values()
    for i, row in enumerate(user_links_history):
        if row[0] == user_id and link in row[1:]:
            user_links_history_sheet.delete_rows(i + 1)
            return


def get_user_links(user_id: str) -> list:
    all_users_links = user_links_history_sheet.get_all_values()  # gets entire table
    for row in all_users_links:
        if row[0] == user_id:
            return row[1:]
    return []

def get_sent_links(user_id: str) -> List[str]:
    all_user_links = user_links_history_sheet.get_all_values()  # gets entire table
    sent_links = []
    for row in all_user_links:
        if row[0] == user_id:
            sent_links.extend(row[1:])  # We start from 2nd column, as 1st is the user's own link
    return sent_links  # No need for ID to URL conversion anymore




def get_random_bot_links(user_id: str, url: str, user_links: List[str], sent_links: List[str], num: int) -> List[str]:
    all_links = get_all_links()  # Получаем все ссылки из базы данных
    unsent_links = list(set(all_links) - set(sent_links))  # Находим ссылки, которые еще не отправлялись пользователю
    unsent_links_id = [get_id_by_link(link) for link in unsent_links]  # Преобразуем эти ссылки в их соответствующие ID

    user_link = get_user_profile_url(user_id)  # Получаем ссылку пользователя
    user_link_id = get_id_by_link(user_link)  # Преобразуем ссылку пользователя в ее соответствующий ID

    if user_link_id in unsent_links_id:  # Если ID ссылки пользователя присутствует в списке непосланных ID, удаляем его
        unsent_links_id.remove(user_link_id)

    if len(unsent_links_id) <= num:  # Если непосланных ссылок меньше или равно запрошенному количеству, возвращаем их все
        return unsent_links_id
    else:  # В противном случае возвращаем запрошенное количество случайных ссылок
        return random.sample(unsent_links_id, num)






def update_user_links_history(user_id: int, sent_links: List[str]) -> None:
    # Get the user link from telegram user id
    user_link = get_user_link(user_id)

    # Get all links from URL list
    all_links = urls_list_sheet.get_all_values()

    # Check if user_link is already in all_links
    if user_link not in [link[1] for link in all_links]:
        # If not, add new user_link to the urls_list_sheet
        urls_list_sheet.insert_row([str(user_id), user_link], len(all_links) + 1)
    
    # Get user_link_id from all_links
    user_link_id = None
    for link in all_links:
        if link[1] == user_link:
            user_link_id = link[0]
            break

    # Get IDs of all sent links
    sent_links_ids = []
    for url in sent_links:
        for link in all_links:
            if link[1] == url:
                sent_links_ids.append(link[0])
                break

    # Create a new row
    new_row = [str(user_id), str(user_link_id)] + [str(id) for id in sent_links_ids]

    # Remove any existing row with the same user_id
    existing_rows = user_links_history_sheet.get_all_values()
    for i, row in enumerate(existing_rows):
        if row[0] == str(user_id):
            user_links_history_sheet.delete_row(i + 1)
            break

    # Insert new_row at the end
    user_links_history_sheet.insert_row(new_row, len(existing_rows) + 1)


    
def update_sent_links_history(user_id: int, bot_links: List[str]) -> None:
    try:
        all_users_links = user_links_history_sheet.get_all_values()  # gets entire table
        # Convert bot_links (URLs) back to IDs
        bot_links_ids = [get_id_by_link(link) for link in bot_links if link != get_link_by_id(user_id)]
        # Combine all IDs into one string, separated by commas
        bot_links_ids_str = ','.join(map(str, bot_links_ids))

        # looking for the user in the history
        for i, row in enumerate(all_users_links):
            if row[0] == str(user_id):
                # updating user row
                row[1] = bot_links_ids_str
                user_links_history_sheet.update('A{}:B{}'.format(i+1, i+1), [row])
                return
        # if the user is not in the history yet
        user_links_history_sheet.insert_row([str(user_id), bot_links_ids_str], len(all_users_links) + 1)
    except Exception as e:
        logging.error(f"Failed to update sent links history: {str(e)}")
        raise e





def main() -> None:
    """Запуск бота."""
    updater = Updater(TELEGRAM_API_TOKEN)

    dispatcher = updater.dispatcher

    dispatcher.add_handler(CommandHandler("start", start))
    dispatcher.add_handler(CommandHandler("add", add))
    dispatcher.add_handler(CommandHandler("delete", delete))
    dispatcher.add_handler(CommandHandler("show_more", show_more))
    dispatcher.add_handler(MessageHandler(Filters.text, process_link))

    updater.start_polling()

    updater.idle()

if __name__ == '__main__':
    main()
